# Spaceship KNN Submission

Hello there whoever handles this aku lupa, ini cuma coba-coba aja si pake KNeighborsClassifier buat test submission. Feel free to modify ya, lov u makasi semangat. -Egar

In [ ]:
!pip install catboost==1.2.5


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install optuna==3.6.1


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler


In [ ]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 19.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import optuna
from optuna.integration import OptunaSearchCV
from sklearn.model_selection import cross_val_score

# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'loss_function': 'CrossEntropy',
        'random_seed': 42,
        'verbose': 0
    }
    
    pipeline = Pipeline([
        ('scaler', MinMaxScaler()), 
        ('classifier', CatBoostClassifier(**param))
    ])

    # Using StratifiedKFold for handling imbalanced data
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')
    
    return scores.mean()

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

# Best parameters
best_params = study.best_params
print("Best parameters:", best_params)

# Train final model with the best parameters
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('classifier', CatBoostClassifier(**best_params))
])
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
pipeline.fit(X_df, y_df)

# Predict probabilities for submission data
y_submission_proba = pipeline.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict.to_csv('bankChurn_catop.csv', index=False)


[I 2024-05-17 12:52:40,176] A new study created in memory with name: no-name-db4f9125-f95a-4983-9087-7dbdcc964dd1
[I 2024-05-17 13:06:04,012] Trial 0 finished with value: 0.8990921480941305 and parameters: {'iterations': 894, 'learning_rate': 0.058912658796255804, 'depth': 4, 'l2_leaf_reg': 5.766225671195117, 'border_count': 43}. Best is trial 0 with value: 0.8990921480941305.
[I 2024-05-17 13:09:43,191] Trial 1 finished with value: 0.8995155151982827 and parameters: {'iterations': 285, 'learning_rate': 0.07395213763173324, 'depth': 6, 'l2_leaf_reg': 8.211278080343835, 'border_count': 81}. Best is trial 1 with value: 0.8995155151982827.
[I 2024-05-17 13:34:32,779] Trial 2 finished with value: 0.8947711394809537 and parameters: {'iterations': 892, 'learning_rate': 0.060987107789580365, 'depth': 10, 'l2_leaf_reg': 9.607751177907897, 'border_count': 116}. Best is trial 1 with value: 0.8995155151982827.
[I 2024-05-17 13:43:20,915] Trial 3 finished with value: 0.8993361878379998 and paramet

# Bring the Data In

In [ ]:
# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Model Training

In [ ]:
# Membuat pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('classifier', CatBoostClassifier(random_seed=42, verbose=0))
])


param_grid = {
    'classifier__iterations': [500, 1000, 1500],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__depth': [6, 8, 10],
    'classifier__l2_leaf_reg': [1, 3, 5, 7],
    'classifier__border_count': [32, 64, 128],
    'classifier__loss_function': ['CrossEntropy', 'Logloss']
}


# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# GridSearchCV setup dengan parameter yang lebih luas
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the model dengan metric tambahan
y_pred = best_model.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))
print("F1-Score on test set:", f1_score(y_test, y_pred))


Fitting 10 folds for each of 648 candidates, totalling 6480 fits


In [ ]:
# Evaluation on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.76      0.57      0.65      6955

    accuracy                           0.87     33007
   macro avg       0.82      0.76      0.79     33007
weighted avg       0.86      0.87      0.86     33007

Confusion Matrix:
[[24790  1262]
 [ 3004  3951]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        <catboost.core.CatBoostClassifier object at 0x7fa5d55dbaf0>)]),
             param_grid={'classifier__border_count': [64],
                         'classifier__depth': [10],
                         'classifier__iterations': [500],
                         'classifier__l2_leaf_reg': [5],
                         'classifier__learning_rate': [0.01],
                         'classifier__loss_function': ['CrossEntropy']},
             scoring='roc_auc', verbose=1)

In [ ]:
# Predict probabilities for submission data
y_submission_proba = grid_search.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

[0.01629698 0.85410913 0.02370677 ... 0.02108736 0.19028542 0.16779655]


,id,Exited
0,165034,0.016297
1,165035,0.854109
2,165036,0.023707
3,165037,0.221638
4,165038,0.378906
...,...,...
110018,275052,0.035880
110019,275053,0.096423
110020,275054,0.021087
110021,275055,0.190285


In [ ]:
submission_dict.to_csv('bankChurn_cat2.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>